# Make input files for Lake Washington simulation

In [ ]:
%matplotlib inline

In [ ]:
from pylab import *
from IPython.display import Image
import os,sys
from clawpack.visclaw import colormaps

In [ ]:
sys.path.insert(0,'../../new_python')
import region_tools, topotools, marching_front, dtopotools, topotools, kmltools
from plottools import pcolorcells

In [ ]:
sea_level_lake = 3.1

In [ ]:
zmin = -60.
zmax = 40.

land_cmap = colormaps.make_colormap({ 0.0:[0.1,0.4,0.0],
                                     0.25:[0.0,1.0,0.0],
                                      0.5:[0.8,1.0,0.5],
                                      1.0:[0.8,0.5,0.2]})

sea_cmap = colormaps.make_colormap({ 0.0:[0,0,1], 1.:[.8,.8,1]})

cmap, norm = colormaps.add_colormaps((land_cmap, sea_cmap),
                                     data_limits=(zmin,zmax),
                                     data_break=sea_level_lake)
                                     
sea_cmap_dry = colormaps.make_colormap({ 0.0:[1.0,0.7,0.7], 1.:[1.0,0.7,0.7]})
cmap_dry, norm_dry = colormaps.add_colormaps((land_cmap, sea_cmap_dry),
                                     data_limits=(zmin,zmax),
                                     data_break=sea_level_lake)

## Download 1/3" topo of Lake Washington

In [ ]:
try:
    # read existing file
    topo_1_3sec = topotools.Topography()
    topo_1_3sec.read('input_files/topo_1_3sec_LakeWA.tt3')
    print('Read existing file')
except:
    try:
        extent = [-122.45, -122.1, 47.39, 47.81]
        path = 'https://www.ngdc.noaa.gov/thredds/dodsC/regional/puget_sound_13_mhw_2014.nc'
        topo_1_3sec = topotools.read_netcdf(path, extent=extent, verbose=True)
        topo_1_3sec.write('input_files/topo_1_3sec_LakeWA.tt3')
    except:
        # in case that fails:
        from clawpack.clawutil.data import get_remote_file
        print('Reading from thredds server failed, instead read cached version from geoclaw...')
        remote_topo_dir = 'http://depts.washington.edu/clawpack/geoclaw/topo/WA/'
        fname = 'topo_1_3sec_LakeWA.tt3'
        path = os.path.join(remote_topo_dir, fname)
        get_remote_file(path, output_dir='input_files', file_name=fname, 
                        verbose=True)
        topo_1_3sec = topotools.Topography()
        topo_1_3sec.read('input_files/topo_1_3sec_LakeWA.tt3') 

## Define boundary between Sound and Lake

This will be used for selecting fgmax points below, and also for setting the initial lake elevation to be greater than the initial sea level (MHW) in the Sound. 

In [ ]:
ybdry = array([47.39,   47.46,  47.54, 47.60,      47.65,     47.68,   47.82])
xbdry = array([-122.2, -122.2, -122.3, -122.31, -122.3974, -122.3974, -122.28])

In [ ]:
figure(figsize=(10,6))
ax = subplot(121)
contourf(topo_1_3sec.X,topo_1_3sec.Y,topo_1_3sec.Z,[0,1e6],colors=[[.5,1,.5]], alpha=0.3)
contour(topo_1_3sec.X,topo_1_3sec.Y,topo_1_3sec.Z,[0],colors='g')
plot(xbdry,ybdry,'k')
gca().set_aspect(1./cos(48*pi/180.));

## dtopo file for "Seattle Fault" event

In [ ]:
# 6 arcsecond resolution for dtopo file:
x = linspace(-122.5, -122.1, 241)
y = linspace(47.4, 47.8, 241)
print("Resolution: %5.2f arcsecond by %5.2f arcsecond" % ((x[1]-x[0])*3600, (y[1]-y[0])*3600))

In [ ]:
s = dtopotools.SubFault()

s.strike = 90.
s.dip = 40.
s.rake = 90.
s.slip = 10.
s.latitude = 47.57
s.longitude = -122.25
s.length = 20e3
s.width = 16e3 / cos(s.dip*pi/180.)
print('width = %.2f m' % s.width)
s.depth = 1.  # meters
s.coordinate_specification = 'top center'
s.calculate_geometry()
f = dtopotools.Fault()
f.subfaults = [s]
dtopo = f.create_dtopography(x,y,times=[1.]);
print('Mw = %6.2f' % f.Mw())

In [ ]:
cmax_dZ = abs(dtopo.dZ).max()
dZ_interval = cmax_dZ / 10.
print('maximum surface displacement = %.2f m' % cmax_dZ)
print('Will plot contours with dZ interval = %.2f m' % dZ_interval)

In [ ]:
xtrans = -122.265
jtrans = where(dtopo.x<xtrans)[0].max()

figure(figsize=(10,6))
ax = subplot(121)
contourf(topo_1_3sec.X,topo_1_3sec.Y,topo_1_3sec.Z,[0,1e6],colors=[[.5,1,.5]], alpha=0.3)
contour(topo_1_3sec.X,topo_1_3sec.Y,topo_1_3sec.Z,[0],colors='g')
f.plot_subfaults(axes=ax)
dtopo.plot_dZ_colors(1., axes=ax, cmax_dZ = cmax_dZ, 
                     dZ_interval=dZ_interval)
plot(xtrans*ones(dtopo.Y[:,jtrans].shape), dtopo.Y[:,jtrans], 'k--')
text(xtrans+0.01, 47.77, 'transect')
axis([-122.5,-122.1,47.4,47.8]);

subplot(122)
plot(dtopo.Y[:,jtrans], dtopo.dZ[-1,:,jtrans])
title('Displacement on transect at longitude x = %7.3f' % (xtrans))
xlabel('Latitude')
ylabel('Seafloor deformation (m)');

### make dtopo file:

In [ ]:
fname = 'input_files/SFfaultLakeWA.tt3'
dtopo.write(fname, dtopo_type=3)

### plot topo on transect:

In [ ]:
jtrans_topo = where(topo_1_3sec.x<xtrans)[0].max()
fill_between(topo_1_3sec.Y[:,jtrans_topo], topo_1_3sec.Z[:,jtrans_topo], 
             -100., color=[.5,1,.6])
plot(topo_1_3sec.Y[:,jtrans_topo], topo_1_3sec.Z[:,jtrans_topo], 'g')
Zzero = where(topo_1_3sec.Z[:,jtrans_topo]<0, 0, nan)
fill_between(topo_1_3sec.Y[:,jtrans_topo], topo_1_3sec.Z[:,jtrans_topo], 
             Zzero, color=[.6,.6,1])
plot(topo_1_3sec.Y[:,jtrans_topo], Zzero, 'b')
xlim(47.5, 47.8)

## Select fgmax points

Convert the boundary xbdry,ybdry between Sound and Lake into a Ruled Rectangle enclosing only the lake:

In [ ]:
rr_lake = region_tools.RuledRectangle()
rr_lake.s = ybdry
rr_lake.lower = xbdry
rr_lake.upper = -122.16 * ones(xbdry.shape)
rr_lake.ixy = 'y'
rr_lake.method = 1
rr_lake_x, rr_lake_y = rr_lake.vertices()  # vertices of polygon
#rr_lake.write('input_files/RuledRectangle_Lake.data') # make a better one below for AMR

Start by selecting points in this ruled rectangle that have elevation below `sea_level_lake` (selecting all points in the lake) and march outward to include a buffer of at least `buffer_width` onshore points regardless of topography:

In [ ]:
topo = topo_1_3sec
mask_out = rr_lake.mask_outside(topo.X, topo.Y)
buffer_width = 10
pts_chosen = marching_front.select_by_flooding(topo.Z, 
                                               mask=mask_out,
                                               Z1=sea_level_lake,
                                               Z2=1e6, 
                                               max_iters=buffer_width) 

Add all points connected to these with `topo < Zmax`:

In [ ]:
Zmax = 15.
pts_chosen = marching_front.select_by_flooding(topo.Z,
                                               mask=mask_out,
                                               Z1=sea_level_lake, 
                                               Z2=Zmax, 
                                               prev_pts_chosen=pts_chosen,
                                               max_iters=None) 

In [ ]:
Zfgmax = ma.masked_array(topo.Z, logical_not(pts_chosen))

In [ ]:
figure(figsize=(6,10))
pcolorcells(topo.X, topo.Y, Zfgmax, cmap=cmap, norm=norm)
colorbar(extend='both')
gca().set_aspect(1./cos(48*pi/180.))
plot(rr_lake_x, rr_lake_y, 'k')
xlim(-122.45,-122.15)
ylim(47.42,47.83);

In [ ]:
fname_fgmax_mask = 'input_files/fgmax_pts_LakeWA.data'
topo_fgmax_mask = topotools.Topography()
topo_fgmax_mask._x = topo.x
topo_fgmax_mask._y = topo.y     
topo_fgmax_mask._Z = where(pts_chosen,1,0)
topo_fgmax_mask.generate_2d_coordinates()

topo_fgmax_mask.write(fname_fgmax_mask, topo_type=3, Z_format='%1i')
print('Created %s' % fname_fgmax_mask)

## Create an AMR flagregion covering the fgmax points

In [ ]:
rr_fgmax = region_tools.ruledrectangle_covering_selected_points(topo.X, topo.Y, 
                                                                pts_chosen, 
                                                                ixy='y', method=0,
                                                                padding=0, verbose=True)

In [ ]:
figure(figsize=(12,6))
xv,yv = rr_fgmax.vertices()
pcolorcells(topo.X, topo.Y, Zfgmax, cmap=cmap, norm=norm)
#axis([-122.47, -122.40, 47.925, 47.965])
gca().set_aspect(1./cos(48*pi/180.))
plot(xv, yv, 'r')
title("With ixy = 'y'");

In [ ]:
fname_flagregion = 'input_files/RuledRectangle_LakeWA.data'
rr_fgmax.write(fname_flagregion)

## Make kml files for viewing on Google Earth

In [ ]:
Zfgmax_land = ma.masked_where(Zfgmax<=sea_level_lake, Zfgmax)
fig,ax,png_extent,kml_dpi = kmltools.pcolorcells_for_kml(topo.x, topo.y, Zfgmax_land,
                                                 png_filename='fgmax_land_for_kml.png',
                                                 dpc=2, cmap=cmap, norm=norm)

In [ ]:
Zfgmax_water = ma.masked_where(Zfgmax>sea_level_lake, Zfgmax)
fig,ax,png_extent,kml_dpi = kmltools.pcolorcells_for_kml(topo.x, topo.y, Zfgmax_water,
                                                 png_filename='fgmax_water_for_kml.png',
                                                 dpc=2, cmap=cmap, norm=norm)

In [ ]:
kmltools.png2kml(topo.extent, png_files=['fgmax_land_for_kml.png', 'fgmax_water_for_kml.png'],
                 png_names=['fgmax land points','fgmax water points'], 
                 name='fgmax_points', fname='fgmax_points.kmz')